In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool



########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location in ["서울", "인천"]:
        return "현재 기온은 20도이고 구름이 많아."
    else:
        return "현재 기온은 30도이며 맑아"
    

@tool
def get_coolest_cities():
    """Get a list of coolest cities"""
    return "서울, 인천"


tools = [get_weather, get_coolest_cities]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합
llm_with_tools = llm.bind_tools(tools)


########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[tool])

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)


In [9]:
# 그래프 빌더 컴파일
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()


graph = graph_builder.compile()

In [ ]:
question = (
    "서울날씨어떄?"
)

for event in graph.stream({"messages": [("user", question)]}):
    for value in event.values():
        value["messages"][-1].pretty_print()

In [29]:
from dotenv import load_dotenv
from langchain_core.tools import tool
from typing import Annotated
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

load_dotenv()


class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]

@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location in ["서울", "인천"]:
        return "현재 기온은 20도이고 구름이 많아."
    else:
        return "현재 기온은 30도이며 맑아"






tools = [get_weather]
tool_node = ToolNode(tools)
model_with_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)


def should_continue(state: State):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: State):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(State)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")


from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()

app = workflow.compile(checkpointer=memory)


In [30]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)


In [34]:
for chunk in app.stream(
    {"messages": [("human", "내가 뭘 좋아한다고?")]}, stream_mode="values", config=config
):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

내가 뭘 좋아한다고?
================================== Ai Message ==================================

당신은 치킨을 좋아한다고 말씀하셨습니다!


In [32]:
prepositions = [
    'about', 'above', 'across', 'after', 'against', 'along', 'among', 'around',
    'at', 'before', 'behind', 'below', 'beneath', 'beside', 'between', 'beyond',
    'by', 'concerning', 'despite', 'down', 'during', 'except', 'for', 'from',
    'in', 'inside', 'into', 'like', 'near', 'of', 'off', 'on', 'onto',
    'out', 'outside', 'over', 'past', 'regarding', 'since', 'through',
    'throughout', 'till', 'to', 'toward', 'under', 'underneath', 'until',
    'up', 'upon', 'with', 'within', 'without'
]

conjunctions = [
    'and', 'but', 'or', 'nor', 'for', 'yet', 'so',
    'both', 'either', 'neither', 'not only', 'but also', 'whether', 'or',
    'after', 'although', 'as', 'because', 'before', 'even if', 'even though',
    'if', 'once', 'since', 'so that', 'than', 'that', 'though', 'unless',
    'until', 'when', 'whenever', 'where', 'whereas', 'wherever', 'whether',
    'while'
]



prepositions_conjunctions_list = prepositions + conjunctions

In [37]:
from langchain.schema import Document
table_info = Document(metadata={}, page_content="client_stream_activated_on_product: 고객이 제품을 활성화할 때 트리거되는 활동 데이터\nColumns:\n id: 이 테이블의 기본 키\nentity_id: 고객의 엔티티 ID\nactivity_ts: 활동이 발생한 타임스탬프\nactivity: 활동의 이름\nrevenue_impact: 활동의 수익 영향\nfeature_json: 활동과 관련된 기능 데이터를 포함하는 JSON 문자열로, 'active_users', 'churn_risk_users', 'churned_users', \n'free_users', 'paid_users', 'grace_period_users', 'canceled_users', 'new_users', 'returning_users', \n'trial_users'와 같은 고객 세그먼트와 'basic_plan', 'standard_plan', 'premium_plan', 'monthly_plan', \n'annual_plan', 'lifetime_plan'과 같은 요금제 유형, 그리고 'chris', 'john', 'jane', 'jim', 'jill', 'james'와 \n같은 CSM 이름, 'tier1', 'tier2', 'tier3', 'tier4', 'tier5'와 같은 MRR 등급 유형이 포함됩니다.\n"), Document(metadata={}, page_content="client_stream_active_on_subscription: 고객이 구독에서 활동 중일 때 트리거되는 활동 데이터\nColumns:\n id: 이 테이블의 기본 키\nentity_id: 고객의 엔티티 ID\nactivity_ts: 활동이 발생한 타임스탬프\nactivity: 활동 이름\nrevenue_impact: 활동의 수익 영향(해당되는 경우)\nfeature_json: 활동과 관련된 기능 데이터를 포함하는 JSON 문자열로, 'active_users', 'churn_risk_users', 'churned_users', 'free_users', 'paid_users', 'grace_period_users', 'canceled_users', 'new_users', 'returning_users', 'trial_users'와 같은 고객 세그먼트 및 'basic_plan', 'standard_plan', 'premium_plan', 'monthly_plan', 'annual_plan', 'lifetime_plan'과 같은 플랜 유형을 포함합니다.")


prefix_count = {}


for table in range(len(table_info)):
    table_name = (table_info[table].page_content.split(':')[0].strip())
    print(table_name)
        


client_stream_activated_on_product
client_stream_active_on_subscription


In [27]:
s = "client_stream_activated_on_product"
s.count('_')



4

In [36]:
from langchain.schema import Document
table_info = Document(metadata={}, page_content="client_stream_activated_on_product: 고객이 제품을 활성화할 때 트리거되는 활동 데이터\nColumns:\n id: 이 테이블의 기본 키\nentity_id: 고객의 엔티티 ID\nactivity_ts: 활동이 발생한 타임스탬프\nactivity: 활동의 이름\nrevenue_impact: 활동의 수익 영향\nfeature_json: 활동과 관련된 기능 데이터를 포함하는 JSON 문자열로, 'active_users', 'churn_risk_users', 'churned_users', \n'free_users', 'paid_users', 'grace_period_users', 'canceled_users', 'new_users', 'returning_users', \n'trial_users'와 같은 고객 세그먼트와 'basic_plan', 'standard_plan', 'premium_plan', 'monthly_plan', \n'annual_plan', 'lifetime_plan'과 같은 요금제 유형, 그리고 'chris', 'john', 'jane', 'jim', 'jill', 'james'와 \n같은 CSM 이름, 'tier1', 'tier2', 'tier3', 'tier4', 'tier5'와 같은 MRR 등급 유형이 포함됩니다.\n"), Document(metadata={}, page_content="client_stream_active_on_subscription: 고객이 구독에서 활동 중일 때 트리거되는 활동 데이터\nColumns:\n id: 이 테이블의 기본 키\nentity_id: 고객의 엔티티 ID\nactivity_ts: 활동이 발생한 타임스탬프\nactivity: 활동 이름\nrevenue_impact: 활동의 수익 영향(해당되는 경우)\nfeature_json: 활동과 관련된 기능 데이터를 포함하는 JSON 문자열로, 'active_users', 'churn_risk_users', 'churned_users', 'free_users', 'paid_users', 'grace_period_users', 'canceled_users', 'new_users', 'returning_users', 'trial_users'와 같은 고객 세그먼트 및 'basic_plan', 'standard_plan', 'premium_plan', 'monthly_plan', 'annual_plan', 'lifetime_plan'과 같은 플랜 유형을 포함합니다.")

# 언더스코어 개수를 저장할 딕셔너리
underscore_count = {}

for table in range(len(table_info)):
    table_name = table_info[table].page_content.split(':')[0].strip()
    # 언더스코어 개수 계산
    count = table_name.count('_')
    # 딕셔너리에 저장
    underscore_count[table_name] = count

print("테이블별 언더스코어 개수:")
for table_name, count in underscore_count.items():
    print(f"{table_name}: {count}개")

테이블별 언더스코어 개수:
client_stream_activated_on_product: 4개
client_stream_active_on_subscription: 4개


In [54]:
from langchain.schema import Document

table_info = [
    Document(metadata={}, page_content="client_stream_activated_on_product: 고객이 제품을 활성화할 때 트리거되는 활동 데이터\nColumns:\n id: 이 테이블의 기본 키\nentity_id: 고객의 엔티티 ID\nactivity_ts: 활동이 발생한 타임스탬프\nactivity: 활동의 이름\nrevenue_impact: 활동의 수익 영향\nfeature_json: 활동과 관련된 기능 데이터를 포함하는 JSON 문자열로, 'active_users', 'churn_risk_users', 'churned_users', \n'free_users', 'paid_users', 'grace_period_users', 'canceled_users', 'new_users', 'returning_users', \n'trial_users'와 같은 고객 세그먼트와 'basic_plan', 'standard_plan', 'premium_plan', 'monthly_plan', \n'annual_plan', 'lifetime_plan'과 같은 요금제 유형, 그리고 'chris', 'john', 'jane', 'jim', 'jill', 'james'와 \n같은 CSM 이름, 'tier1', 'tier2', 'tier3', 'tier4', 'tier5'와 같은 MRR 등급 유형이 포함됩니다.\n"), Document(metadata={}, page_content="client_stream_active_on_subscription: 고객이 구독에서 활동 중일 때 트리거되는 활동 데이터\nColumns:\n id: 이 테이블의 기본 키\nentity_id: 고객의 엔티티 ID\nactivity_ts: 활동이 발생한 타임스탬프\nactivity: 활동 이름\nrevenue_impact: 활동의 수익 영향(해당되는 경우)\nfeature_json: 활동과 관련된 기능 데이터를 포함하는 JSON 문자열로, 'active_users', 'churn_risk_users', 'churned_users', 'free_users', 'paid_users', 'grace_period_users', 'canceled_users', 'new_users', 'returning_users', 'trial_users'와 같은 고객 세그먼트 및 'basic_plan', 'standard_plan', 'premium_plan', 'monthly_plan', 'annual_plan', 'lifetime_plan'과 같은 플랜 유형을 포함합니다.")# 테이블이 더 있다고 가정
]

# 테이블별로 분리된 이름을 저장할 리스트들의 리스트
split_table_names = []
for table in table_info:
    table_name = table.page_content.split(':')[0].strip()
    split_names = table_name.split('_')
    split_table_names.append(split_names)
    

# 가장 긴 테이블 이름의 길이 구하기
max_len = max(len(names) for names in split_table_names)


# # 각 위치별 리스트 만들기
result_lists = [[] for _ in range(max_len)]


for names in split_table_names:
    for idx in range(max_len):
        # 해당 인덱스가 있으면 추가, 없으면 None 또는 '' 등으로 채움
        if idx < len(names):
            if names[idx] not in prepositions_conjunctions_list:
                result_lists[idx].append(names[idx])
            # else:
            #     result_lists[idx].append('')
                
print(len(result_lists))     # 혹은 None

# 결과 출력
ordinal_names = [
    "first", "second", "third", "fourth", "fifth",
    "sixth", "seventh", "eighth", "ninth", "tenth",
    "eleventh", "twelfth", "thirteenth", "fourteenth", "fifteenth"
]

# 자연스러운 변수명으로 딕셔너리에 저장
named_lists = {}
for idx, lst in enumerate(result_lists):
    if idx < len(ordinal_names):
        var_name = f"{ordinal_names[idx]}_list"
    else:
        var_name = f"{idx+1}_list"
    named_lists[var_name] = lst


print(named_lists)

5
{'first_list': ['client', 'client'], 'second_list': ['stream', 'stream'], 'third_list': ['activated', 'active'], 'fourth_list': [], 'fifth_list': ['product', 'subscription']}
